In [1]:
%%capture
#@title Setup Environment
# Install the latest Tensorflow version.
!pip install "tensorflow-text==2.11.*"
!pip install bokeh
!pip install simpleneighbors[annoy]
!pip install tqdm

In [ ]:
import numpy
numpy.version.version

'1.22.4'

In [2]:
#@title Setup common imports and functions
import bokeh
import bokeh.models
import bokeh.plotting
import numpy as np
import os
import pandas as pd
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import sklearn.metrics.pairwise




In [3]:
# The 16-language multilingual module is the default but feel free
# to pick others from the list and compare the results.
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3' #@param ['https://tfhub.dev/google/universal-sentence-encoder-multilingual/3', 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3']

model = hub.load(module_url)

def embed_text(input):
  return model(input)

In [13]:
import json
import pandas as pd

def get_data_embeds(questions):
  embeds = {
      text: embed_text(text).numpy().tolist() for text in questions
  } # {Question: embedding of the question}
  return embeds

def calc_embeds(file_path, output_path):
  df = pd.read_csv(file_path, sep="\t")
  q_df = df.question.values
  df_embeds = get_data_embeds(q_df)
  with open(output_path, 'w') as f:
    json.dump(df_embeds,f)

calc_embeds(r"/content/drive/MyDrive/University/Internships/CheckScan/AI chatbot/data/new_df.csv", "new_df_question.json")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity




def get_answer(question, embeds):
  '''
    This function will try to get the 3 most relevent answers by comparing the question with similar questions
  '''
  question_embeding = embed_text(question).numpy() # getting the embedings of the question
  question_embeding = question_embeding.reshape(1, -1)
  similarity_scores = []
  for text, embed in list(embeds.items()):
      embed = np.array(embed).reshape(1, -1)
      similarity_score = cosine_similarity(question_embeding, embed)[0][0]
      similarity_scores.append(similarity_score)
  top_indices = np.argsort(similarity_scores)[-3:][::-1]
  top_questions = [list(embeds.keys())[index] for index in top_indices]
  top_answers = [data.answer.values[index] for index in top_indices]
  for i in range(3):
    print(top_questions[i])
    print(top_answers[i])
    print("--------------")


question = ['пополнение мир регистрироваться чек день пополнить почти время долгий получаться карта каждый происходить сбербанк этот не']
get_answer(question, embeds)

пополнение мир регистрироваться чек день пополнить почти время долгий получаться карта каждый происходить сбербанк этот не
Пришлите, пожалуйста, Ваш пригласительный код - он указан на странице профиля в приложении. Просто скопируйте его, нажав на соответствующую кнопку и вставьте его нам в ответном письме.  
--------------
поменять дать карта мир сбербанк
   Если на балансе карты еще остались средства, то вывести средства с карты вы можете следующими способами:    1. Оплатить картой товары или услуги в Интернете - для этого необходимо ввести реквизиты карты со страницы баланса (кнопка "Реквизиты") в поле оплаты. Если требуется ввести имя владельца карты - впишите имя и фамилию на латинице;    2. Вывести средства на счёт мобильного телефона - для этого необходимо открыть приложение мобильного оператора, счёт которого хотите пополнить, нажать "Пополнить" и ввести реквизиты карты для оплаты. Пополнение проходит без комиссии;    3. Лично обратиться в отделение банка для расторжения договор

In [ ]:
import numpy as np
import pandas as pd
# import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import sklearn.metrics.pairwise
import json
from sklearn.metrics.pairwise import cosine_similarity


class Model:
    def __init__(self) -> None:
        module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3'
        self.model = hub.load(module_url)
        self.embeds = self.load_embeds()
        self.data = pd.read_csv(r'/content/drive/MyDrive/University/Internships/CheckScan/AI chatbot/train_msgs.csv', sep='\t')

    def load_embeds(self):
        with open('train_embeds.json', 'r') as f:
            embeds = json.load(f)
        return embeds

    def embed_text(self, text):
        """
            returning the embeddings of an input text
        """
        return self.model(text)

    def get_top_ans(self, question, n_answer=3):
        """
            This function will try to get the 3 most relevant answers by comparing the question with similar questions.
            The similarity is calculated using cosine_similarity
        """
        question_embedding = self.embed_text(
            question).numpy()  # getting the embeddings of the question
        question_embedding = question_embedding.reshape(1, -1)
        similarity_scores = []
        for _, embed in list(self.embeds.items()):
            embed = np.array(embed).reshape(1, -1)
            similarity_score = cosine_similarity(
                question_embedding, embed)[0][0]
            similarity_scores.append(similarity_score)
        top_indices = np.argsort(similarity_scores)[-3:][::-1]
        top_questions = [list(self.embeds.keys())[index]
                         for index in top_indices]
        top_answers = [self.data.answer.values[index] for index in top_indices]

        return top_questions[:n_answer], top_answers[:n_answer]

model = Model()
question = [
    'тинькофф карта зачем для деньга если вывод оформлять иметься']
a, b = model.get_top_ans(question)

for i in range(3):
    print(a[i])
    print(b[i])
    print('-'*100)

тинькофф карта зачем для деньга если вывод оформлять иметься
Здравствуйте, к сожалению, нет. Смена номера телефона - вопрос безопасности, на данный момент его можно сменить только при личном посещении банка.
----------------------------------------------------------------------------------------------------
тинькофф оформить она приложение карта вывести деньга через на
Добрый день! Спасибо, что написали.  Информация о проблеме с балансом уже была передана банку-партнёру. Вопрос решаем. Просим отслеживать обновление баланса.  Рекомендуем ориентироваться на баланс, который отображается в смс-сообщениях от банка.  Приносим извинения за доставленные неудобства и надеемся на ваше понимание ?  
----------------------------------------------------------------------------------------------------
не хотеть чекскан получаться тинькофф оформить вывести карта деньга
Здравствуйте, Надежда. Пришлите, пожалуйста, Ваш пригласительный код - он указан на странице профиля в приложении. Просто скопируйте 